# Linear regression using PyTorch built-ins

torch.nn contains utility classes for building neural networks.

In [5]:
import torch.nn as nn
import torch
import numpy as np

In [6]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [7]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

## Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [8]:
from torch.utils.data import TensorDataset

In [9]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [10]:
from torch.utils.data import DataLoader

In [11]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

In [12]:
# We can use dl in a `for` loop
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [ 73.,  66.,  44.],
        [ 68.,  96.,  71.],
        [102.,  43.,  37.]])
tensor([[ 80., 102.],
        [118., 132.],
        [ 57.,  69.],
        [104., 118.],
        [ 22.,  37.]])


In each iteration, the data loader returns one batch of data with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [13]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.5047,  0.4808,  0.0286],
        [ 0.5193, -0.2499,  0.4662]], requires_grad=True)
Parameter containing:
tensor([0.0772, 0.1944], requires_grad=True)


PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [14]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.5047,  0.4808,  0.0286],
         [ 0.5193, -0.2499,  0.4662]], requires_grad=True),
 Parameter containing:
 tensor([0.0772, 0.1944], requires_grad=True)]

In [15]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ -3.3224,  41.4052],
        [ -1.7100,  55.2945],
        [ 22.2535,  38.9253],
        [-29.6679,  59.6644],
        [ 13.4102,  44.6684],
        [ -4.3078,  42.1743],
        [ -2.1622,  56.0106],
        [ 21.7773,  39.9107],
        [-28.6824,  58.8953],
        [ 13.9435,  44.6153],
        [ -3.7746,  42.1213],
        [ -2.6955,  56.0636],
        [ 22.7057,  38.2092],
        [-30.2011,  59.7175],
        [ 14.3957,  43.8992]], grad_fn=<AddmmBackward0>)

## Loss Function

Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [16]:
# Import nn.functional
import torch.nn.functional as F

The `nn.functional` package contains many useful loss and several other utilities.

In [17]:
# Define loss function
loss_fn = F.mse_loss

In [18]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(4797.6455, grad_fn=<MseLossBackward0>)


## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD is short for "stochastic gradient descent". The term _stochastic_ indicates that samples are selected in random batches instead of as a single group.

In [19]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that `model.parameters()` is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

## Train the model

We are now ready to train the model. We'll follow the same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function `fit` that trains the model for a given number of epochs.

In [20]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    # Repeat for given number of epochs
    for epoch in range(num_epochs):

        # Train with batches of data
        for xb, yb in train_dl:

            # 1. Generate predictions
            pred = model(xb)

            # 2. Calculate loss
            loss = loss_fn(pred, yb)

            # 3. Compute gradients
            loss.backward()

            # 4. Update parameters using gradients
            opt.step()

            # 5. Reset the gradients to zero
            opt.zero_grad()

        # Print the progress
        if (epoch+1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")

Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update and `opt.zero_grad` to reset the gradients to zero.

* We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. `loss.item` returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [21]:
fit(100, model, loss_fn=loss_fn, opt=opt, train_dl=train_dl)

Epoch [10/100], Loss: 351.96356201171875
Epoch [20/100], Loss: 117.91170501708984
Epoch [30/100], Loss: 206.84335327148438
Epoch [40/100], Loss: 129.63870239257812
Epoch [50/100], Loss: 92.45074462890625
Epoch [60/100], Loss: 27.72762680053711
Epoch [70/100], Loss: 36.0047607421875
Epoch [80/100], Loss: 49.272560119628906
Epoch [90/100], Loss: 20.238117218017578
Epoch [100/100], Loss: 26.237506866455078


In [22]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 57.4174,  71.9321],
        [ 79.8346, 100.6498],
        [122.1384, 129.5335],
        [ 23.4395,  46.8352],
        [ 96.5450, 113.4719],
        [ 56.1638,  71.0989],
        [ 79.2809, 100.6862],
        [122.2180, 130.1684],
        [ 24.6932,  47.6684],
        [ 97.2450, 114.3416],
        [ 56.8637,  71.9685],
        [ 78.5809,  99.8166],
        [122.6921, 129.4970],
        [ 22.7396,  45.9655],
        [ 97.7987, 114.3051]], grad_fn=<AddmmBackward0>)

In [23]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

Indeed, the predictions are quite close to our targets. We have a trained a reasonably good model to predict crop yields for apples and oranges by looking at the average temperature, rainfall, and humidity in a region. We can use it to make predictions of crop yields for new regions by passing a batch containing a single row of input.

In [24]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[53.4128, 69.5863]], grad_fn=<AddmmBackward0>)

The predicted yield of apples is 54.3 tons per hectare, and that of oranges is 68.3 tons per hectare.